In [1]:
import os
import pandas as pd

CSV_DIR = os.path.join(os.curdir, "dataset", "sorted")
csv_filenames = sorted(os.listdir(CSV_DIR))
csv_filenames


['BASEL.csv',
 'BUDAPEST.csv',
 'DE.csv',
 'DRESDEN.csv',
 'DUSSELDORF.csv',
 'HEATHROW.csv',
 'KASSEL.csv',
 'LJUBLJANA.csv',
 'MAASTRICHT.csv',
 'MALMO.csv',
 'MONTELIMAR.csv',
 'MUENCHEN.csv',
 'OSLO.csv',
 'PERPIGNAN.csv',
 'ROMA.csv',
 'SONNBLICK.csv',
 'STOCKHOLM.csv',
 'TOURS.csv']

In [2]:
csv_dataset = [
    pd.read_csv(os.path.join(CSV_DIR, filename)) for filename in csv_filenames
]
print(len(csv_dataset))
csv_dataset[1]


18


,DATE,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_max
0,20000101,0.92,1.0268,0.52,0.00,3.7,-4.9,-0.7
1,20000102,0.94,1.0297,0.14,0.00,0.4,-3.6,-1.9
2,20000103,0.95,1.0295,0.19,0.00,0.0,-0.8,1.1
3,20000104,0.94,1.0252,0.21,0.00,0.0,-1.0,0.1
4,20000105,0.88,1.0235,0.43,0.00,0.8,0.2,3.9
...,...,...,...,...,...,...,...,...
3649,20091228,0.61,1.0072,0.65,0.00,6.1,3.5,7.0
3650,20091229,0.59,1.0113,0.68,0.32,6.7,3.2,6.9
3651,20091230,0.93,1.0040,0.18,0.12,0.0,1.8,3.5
3652,20091231,0.90,0.9984,0.30,0.69,0.6,4.7,7.4


In [3]:
import tensorflow as tf
import random
import numpy as np
import os

SEED = 1998
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
tf.random.set_seed(SEED)


2022-11-15 16:37:04.878778: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-15 16:37:05.162041: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-15 16:37:05.817628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-15 16:37:05.817747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [4]:
TRAIN_END_INDEX = 2922
VALIDATION_END_INDEX = 3288


def get_dataset(
    dataset: pd.DataFrame,
    sequence_length,
    batch_size,
    train_end_index,
    validation_end_index,
):
    data = dataset
    for col in dataset.columns:
        if "temp_mean" in col:
            target = pd.DataFrame(dataset[col])
            data = data.drop(columns=["DATE"])
            break
    scaled_data = (data - data.mean(axis=0)) / data.std(axis=0)
    train_dataset = tf.keras.utils.timeseries_dataset_from_array(
        scaled_data[:-sequence_length],
        targets=target[sequence_length:],
        sequence_length=sequence_length,
        batch_size=batch_size,
        start_index=0,
        end_index=train_end_index,
    )
    validation_dataset = tf.keras.utils.timeseries_dataset_from_array(
        scaled_data[:-sequence_length],
        targets=target[sequence_length:],
        sequence_length=sequence_length,
        batch_size=batch_size,
        start_index=train_end_index,
        end_index=validation_end_index,
    )
    test_dataset = tf.keras.utils.timeseries_dataset_from_array(
        scaled_data[:-sequence_length],
        targets=target[sequence_length:],
        sequence_length=sequence_length,
        batch_size=batch_size,
        start_index=validation_end_index,
    )
    return (train_dataset, validation_dataset, test_dataset)


sequence_length = 6

batch_dataset = [
    get_dataset(csv_data, sequence_length, 32, TRAIN_END_INDEX, VALIDATION_END_INDEX)
    for csv_data in csv_dataset
]

print(len(batch_dataset))
print(batch_dataset[1])
for data, label in batch_dataset[0][0]:
    print(data[0])
    print(label[0])
    break


2022-11-15 16:37:06.621225: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:42:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-15 16:37:06.632744: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:42:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-15 16:37:06.633571: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:42:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-15 16:37:06.634813: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

18
(<BatchDataset element_spec=(TensorSpec(shape=(None, None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>, <BatchDataset element_spec=(TensorSpec(shape=(None, None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>, <BatchDataset element_spec=(TensorSpec(shape=(None, None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>)
tf.Tensor(
[[ 1.34424888  1.34682202 -1.20851298 -0.38199123 -1.07646018 -1.09549112
  -0.80998751 -1.33429094]
 [ 1.15869866  1.7487146  -1.15505695 -0.43793346 -1.07646018 -1.00108475
  -0.64465741 -1.23109558]
 [ 0.602048    1.69847803 -0.8877768  -0.43793346 -0.22197883 -1.18989749
  -1.03543765 -1.23109558]
 [ 0.41649778  1.04540259 -0.74879112  0.2147259   0.51703206 -0.96062487
  -0.97531762 -0.92150951]
 [ 1.43702399  0.8444563  -0.87708559 -0.30740159 -0.22197883 -0.67740576
  -0.47932731 -0.79538185]
 [ 0.97314844  0.8193380

In [6]:
def get_feature_lengths(batch_dataset):
    feature_lengths = []
    for batch_data in batch_dataset:
        for data, _ in batch_data[0]:
            feature_lengths.append(data.shape[2])
            break
    return feature_lengths

def get_simple_lstm(sequence_length, feature_length, name):
    model_in = tf.keras.Input((sequence_length, feature_length))
    x = tf.keras.layers.LSTM(16, return_sequences=True)(model_in)
    x = tf.keras.layers.LSTM(16)(x)
    model_out = tf.keras.layers.Dense(1)(x)
    return tf.keras.Model(inputs=[model_in], outputs=[model_out], name=name)

feature_lengths = get_feature_lengths(batch_dataset)

models = [
    get_simple_lstm(sequence_length, feature_length, filename.split(".")[0] + "_model")
    for filename, feature_length in zip(csv_filenames, feature_lengths)
]
models[1].summary()


Model: "BUDAPEST_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 6, 7)]            0         
                                                                 
 lstm_2 (LSTM)               (None, 6, 16)             1536      
                                                                 
 lstm_3 (LSTM)               (None, 16)                2112      
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3,665
Trainable params: 3,665
Non-trainable params: 0
_________________________________________________________________


In [7]:
for model in models:
    model.compile("rmsprop", "MSE", ["MAE"])

In [8]:
for filename in csv_filenames:
    os.makedirs(os.path.join(os.curdir, "model", filename.split(".")[0]), exist_ok=True)


In [9]:
max_epoch = 200
histories = [
    model.fit(
        batch_data[0],
        epochs=max_epoch,
        validation_data=batch_data[1],
        callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=5),
            tf.keras.callbacks.ModelCheckpoint(
                os.path.join(os.curdir, "model", filename.split(".")[0]), save_best_only=True
            ),
        ],
    )
    for model, batch_data, filename in zip(models, batch_dataset, csv_filenames)
]


Epoch 1/200


2022-11-15 16:37:44.046373: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


90/92 [============================>.] - ETA: 0s - loss: 152.8053 - MAE: 10.6589

INFO:tensorflow:Assets written to: ./model/BASEL/assets


INFO:tensorflow:Assets written to: ./model/BASEL/assets


...

INFO:tensorflow:Assets written to: ./model/TOURS/assets


92/92 [==============================] - 9s 99ms/step - loss: 4.8006 - MAE: 1.6968 - val_loss: 3.9988 - val_MAE: 1.6001
Epoch 22/200
90/92 [============================>.] - ETA: 0s - loss: 4.6235 - MAE: 1.6684

INFO:tensorflow:Assets written to: ./model/TOURS/assets


INFO:tensorflow:Assets written to: ./model/TOURS/assets


92/92 [==============================] - 8s 85ms/step - loss: 4.6155 - MAE: 1.6681 - val_loss: 3.9956 - val_MAE: 1.6014
Epoch 23/200
92/92 [==============================] - 2s 20ms/step - loss: 4.4648 - MAE: 1.6447 - val_loss: 3.9981 - val_MAE: 1.6028
Epoch 24/200
92/92 [==============================] - 2s 20ms/step - loss: 4.3403 - MAE: 1.6253 - val_loss: 4.0030 - val_MAE: 1.6056
Epoch 25/200
92/92 [==============================] - 2s 20ms/step - loss: 4.2365 - MAE: 1.6088 - val_loss: 4.0090 - val_MAE: 1.6081
Epoch 26/200
92/92 [==============================] - 2s 19ms/step - loss: 4.1491 - MAE: 1.5944 - val_loss: 4.0152 - val_MAE: 1.6106
Epoch 27/200
92/92 [==============================] - 2s 19ms/step - loss: 4.0749 - MAE: 1.5819 - val_loss: 4.0216 - val_MAE: 1.6128
